## Python使用tensorflow读取csv训练DNN模型

本次分享到要点：
1. 使用tf.data.experimental.make_csv_dataset函数，直接将csv数据读取到tf.data.Dataset
2. 使用tf.feature_column.categorical_column_with_vocabulary_list，可以设置怎样读取类别特征；
3. 使用tf.feature_column.numeric_column函数，可以设置怎样读取数字特征；
4. 使用tf.keras.Sequential，可以搭建一个keras的dnn模型；
5. 对于keras的model，可以用model.fit进行训练，使用model.evaluate进行准确率评估，使用model.predict进行新样本预测

### 1. 导入所需的包

In [2]:
import functools

import numpy as np
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf

In [3]:
tf.__version__

'2.0.0'

In [4]:
train_file_path = "./datas/titanic/train.csv"
test_file_path = "./datas/titanic/test.csv"

### 2. 加载数据

In [5]:
# 标签列
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [6]:
def get_dataset(file_path):
    """
    构建tensorflow的数据集格式
    """
    dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=12,
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True)
    return dataset

# 将train和test的csv，分别加载成tensorflow的对象的格式
raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


In [7]:
# 测试一个批次
examples, labels = next(iter(raw_train_data))
print("EXAMPLES: \n", examples, "\n")
print("LABELS: \n", labels)

EXAMPLES: 
 OrderedDict([('sex', <tf.Tensor: id=176, shape=(12,), dtype=string, numpy=
array([b'male', b'female', b'female', b'female', b'female', b'male',
       b'male', b'male', b'female', b'female', b'female', b'male'],
      dtype=object)>), ('age', <tf.Tensor: id=168, shape=(12,), dtype=float32, numpy=
array([35., 31., 44.,  4., 31., 24., 22., 28., 28.,  3., 18., 33.],
      dtype=float32)>), ('n_siblings_spouses', <tf.Tensor: id=174, shape=(12,), dtype=int32, numpy=array([0, 1, 0, 1, 1, 0, 0, 8, 0, 1, 2, 1], dtype=int32)>), ('parch', <tf.Tensor: id=175, shape=(12,), dtype=int32, numpy=array([0, 0, 0, 1, 1, 0, 0, 2, 0, 2, 2, 1], dtype=int32)>), ('fare', <tf.Tensor: id=173, shape=(12,), dtype=float32, numpy=
array([  7.896,  18.   ,  27.721,  23.   ,  26.25 ,   7.496,   9.   ,
        69.55 ,  33.   ,  41.579, 262.375,  20.525], dtype=float32)>), ('class', <tf.Tensor: id=170, shape=(12,), dtype=string, numpy=
array([b'Third', b'Third', b'First', b'Second', b'Second', b'Third',
   

### 3. 数据预处理

机器学习模型的输入，只能是数字

#### 分类数据

In [8]:
# 分类数据的码表
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [9]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
    # 提供码表的特征输入
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [10]:
# 分类特征列
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

#### 连续数据

In [11]:
def process_continuous_data(mean, data):
    # 标准化数据的函数
    data = tf.cast(data, tf.float32) * 1/(2*mean)
    return tf.reshape(data, [-1, 1])

In [12]:
# 提前算好的均值
MEANS = {
    'age' : 29.631308,
    'n_siblings_spouses' : 0.545455,
    'parch' : 0.379585,
    'fare' : 34.385399
}

numerical_columns = []

for feature in MEANS.keys():
    num_col = tf.feature_column.numeric_column(
        feature, normalizer_fn=functools.partial(process_continuous_data, MEANS[feature]))
    numerical_columns.append(num_col)

In [13]:
# 连续特征列的列表
numerical_columns

[NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x7fc1f2700e60>, 29.631308)),
 NumericColumn(key='n_siblings_spouses', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x7fc1f2700e60>, 0.545455)),
 NumericColumn(key='parch', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x7fc1f2700e60>, 0.379585)),
 NumericColumn(key='fare', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function process_continuous_data at 0x7fc1f2700e60>, 34.385399))]

### 4. 构建模型

#### 创建输入层layer

将这两个特征列的集合相加，并且传给 `tf.keras.layers.DenseFeatures` 从而创建一个进行预处理的输入层。

In [14]:
preprocessing_layer = tf.keras.layers.DenseFeatures(
    categorical_columns+numerical_columns)

从 `preprocessing_layer` 开始构建 `tf.keras.Sequential`。

In [15]:
# 构建一个DNN模型h(g(f(x)))
model = tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

In [16]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

### 5. 训练、评估和预测

现在可以实例化和训练模型。

In [17]:
train_data = raw_train_data.shuffle(500)
test_data = raw_test_data

In [18]:
model.fit(train_data, epochs=20)

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Epoch 1/20
53/53 [==============================] - 2s 32ms/step - loss: 0.5944 - accuracy: 0.7177
Epoch 2/20
53/53 [==============================] - 0s 5ms/step - loss: 0.4505 - accuracy: 0.7990
Epoch 3/20
53/53 [==============================] - 0s 4ms/step - loss: 0.4437 - accuracy: 0.8086
Epoch 4/20
53/53 [==============================] - 0s 4ms/step - loss: 0.4136 - accuracy: 0.8198
Epoch 5/20
53/53 [==============================] - 0s 4ms/step - loss: 0.4130 - accuracy: 0.8150
Epoch 6/20
53/53 [==============================] - 0s 5ms/step - loss: 0.3994 - accuracy: 0.8246
Epoch 7/20
53/53 [==============================] - 0s 5ms/step - loss: 0.3983 - accuracy: 0.8293
Epoch 8/20
53/53 [==============================] - 0s 4ms/step -

In [24]:
model.fit?

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_features (DenseFeature multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  1600      
_________________________________________________________________
dense_1 (Dense)              multiple                  2080      
_________________________________________________________________
dense_2 (Dense)              multiple                  528       
_________________________________________________________________
dense_3 (Dense)              multiple                  17        
Total params: 4,225
Trainable params: 4,225
Non-trainable params: 0
_________________________________________________________________


当模型训练完成的时候，你可以在测试集 `test_data` 上检查准确性。

In [20]:
test_loss, test_accuracy = model.evaluate(test_data)

print()
print(f'Test Loss {test_loss}, Test Accuracy {test_accuracy}')

22/22 [==============================] - 1s 27ms/step - loss: 0.4375 - accuracy: 0.8068

Test Loss 0.43752676384015515, Test Accuracy 0.8068181872367859


使用 `tf.keras.Model.predict` 推断一个批次或多个批次的标签。

In [21]:
predictions = model.predict(test_data)
predictions[:10]

array([[0.964],
       [0.092],
       [0.097],
       [0.147],
       [0.115],
       [0.222],
       [0.936],
       [0.509],
       [0.83 ],
       [0.051]], dtype=float32)

In [22]:
list(test_data)[0][1]

<tf.Tensor: id=5931, shape=(12,), dtype=int32, numpy=array([0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0], dtype=int32)>

In [23]:
# 显示部分结果
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
    is_survived = "SURVIVED" if bool(survived) else "DIED"
    print(f"预测活着的概率: {prediction[0]} | 实际值: {is_survived}")

预测活着的概率: 0.964362382888794 | 实际值: DIED
预测活着的概率: 0.0921531617641449 | 实际值: DIED
预测活着的概率: 0.0965917706489563 | 实际值: DIED
预测活着的概率: 0.14703363180160522 | 实际值: DIED
预测活着的概率: 0.11535680294036865 | 实际值: SURVIVED
预测活着的概率: 0.2217859923839569 | 实际值: SURVIVED
预测活着的概率: 0.9363394975662231 | 实际值: SURVIVED
预测活着的概率: 0.5085902810096741 | 实际值: DIED
预测活着的概率: 0.829928994178772 | 实际值: DIED
预测活着的概率: 0.05141010880470276 | 实际值: DIED
